In [ ]:
import cPickle as pickle
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
import CAM_NWB as cn

%matplotlib inline

In [ ]:
path = ('/Volumes/Brain2015/CAM/') # change path to where the CAM data is
pklfile = open(path+'CAM_Meta.pkl','r') # importing the compiled CAM meta data
CAM_Meta = pickle.load(pklfile)
pklfile.close()
CAM_Meta.columns

In [ ]:
grouping = 'specimen' #pick out the grouping you are interested in looking at (ie. HVA, depth etc)
groups= list(set(CAM_Meta[grouping]) )
print groups

In [ ]:
group_of_interest = '185282' # select subgroup that you want to pull out (ie if you selected 'depth'
                             # then you may want to look only at depths of 175)
    
subgroup = CAM_Meta[(CAM_Meta[grouping]==group_of_interest)]  
subgroup_ids = subgroup.lims_id # get the file ids for the subgroup of interest
print len(subgroup_ids) # this will be the number of files to pull out

In [ ]:
"""this is the meat of it. going through each file (x), extracting the raw fluorescence
extracting the stimulus_table to get the number of trials. Then go through each roi or
cell/neuron, for each roi go through each stimulus presentation and window the continuous
rawF trace the length of the intersweep before 'start' through the stimulus presentation
to the length of the intersweep after the end of the 'trail'. The 'short' variable accounts
for instances when the length of the trace is off by a sample due to sampling misalignment
between the rawF and stimuli acquisition. This creates the 3-d numpy array that is 
(roi, trial, samples) in size. This then gets appended to the traces list, so this is
a list of arrays. It prints out which file you're on and it's shape just to track what's
happening"""

traces =[]
sweeplength = 60 
intersweep = 30

for x, filename in enumerate(subgroup_ids):
    print x
    fnm = (path + filename + '/' + filename + '.nwb')
    timestamps,rawF = cn.getFluorescenceTraces(fnm)
    stimparams = cn.getStimulusTable(fnm)
    temptraces = np.zeros((len(rawF),len(stimparams),(sweeplength+2*intersweep)))
    for roi in range(len(rawF)):
        for trial in range(len(stimparams)):
            startsweep = stimparams.start[trial]-intersweep
            endsweep = stimparams.start[trial]+sweeplength+intersweep
            roi_trace = rawF[roi,startsweep:endsweep]
            short = temptraces.shape[2]-(roi_trace.shape[0])   
            if short>0:
                missing_data = np.zeros(short)
                roi_trace = np.append(roi_trace,missing_data)
            temptraces[roi,trial,:] = roi_trace
    print temptraces.shape
    traces.append(temptraces)
   

In [ ]:
np.save(path+'traces_' + group_of_interest + '.npy',traces) # save the file as a numpy arrays